<a href="https://colab.research.google.com/github/ncai25/reddit-depression/blob/main/Reddit_Depression_Stencil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reddit Depression Final Project
Link to the paper: https://dl.acm.org/doi/pdf/10.1145/3578503.3583621

## Preprocessing

In [ ]:
!pip install --upgrade pandas
!pip install gensim

import pickle
import os
from os.path import exists
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier


from gensim.corpora import Dictionary
# from gensim.models import LdaMulticore
from gensim.models import LdaModel

from google.colab import drive
drive.mount('/content/drive')

FILEPATH = 'drive/MyDrive/Colab Notebooks/final_project/student.pkl'
FOLDER = 'drive/MyDrive/Colab Notebooks/final_project'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Data Loading

In [ ]:
def load():
    """Load pickles"""
    if exists(FILEPATH):
        with open(FILEPATH, 'rb') as f:
            data = pd.read_pickle(f)
        return data
    else:
        print(f"File {FILEPATH} does not exist.")
        return None

df = load()

In [ ]:
# List of depression subreddits in the paper
depression_subreddits = ["Anger",
    "anhedonia", "DeadBedrooms",
    "Anxiety", "AnxietyDepression", "HealthAnxiety", "PanicAttack",
    "DecisionMaking", "shouldi",
    "bingeeating", "BingeEatingDisorder", "EatingDisorders", "eating_disorders", "EDAnonymous",
    "chronicfatigue", "Fatigue",
    "ForeverAlone", "lonely",
    "cry", "grief", "sad", "Sadness",
    "AvPD", "SelfHate", "selfhelp", "socialanxiety", "whatsbotheringyou",
    "insomnia", "sleep",
    "cfs", "ChronicPain", "Constipation", "EssentialTremor", "headaches", "ibs", "tinnitus",
    "AdultSelfHarm", "selfharm", "SuicideWatch",
    "Guilt", "Pessimism", "selfhelp", "whatsbotheringyou"
]

In [ ]:
def dataset_generation(df, depression_subreddits):
    """Build control and symptom datasets
    For the control dataset, only posts were at least 180 days earlier than their (earliest) post in a mental health subreddit.”
    """
    # a sympton dataset consists of all the posts from depression subreddits
    df_depression = df[df['subreddit'].isin(depression_subreddits)]

    # a control dataset posts from non-depression subreddits of the same authors
    authors = df_depression['author'].unique()
    df_control = df[(df['author'].isin(authors)) & (~df['subreddit'].isin(depression_subreddits))]

    # find the earliest depression posts for each author
    grouped_author = df_depression.groupby('author')['created_utc'].min()
    author_to_earliest = grouped_author.to_dict()
    time_delta = 180 * 24 * 60 * 60

    # filter the control posts so that they occur at least 180 days before that timestamp
    df_control = df_control.copy()
    df_control['earliest'] = df_control['author'].map(author_to_earliest)
    df_control['delta'] = df_control['created_utc'] - df_control['earliest']
    df_control = df_control[df_control['delta'] < -time_delta]

    return df_depression, df_control

df_depression, df_control = dataset_generation(df, depression_subreddits)

In [ ]:
# Tests, reference (depression_df: 95916; control_df: 4369)
print(len(df_depression))
print(len(df_control))

94514
4369


In [ ]:
# Tokenization + Stopword Removal

!pip install happiestfuntokenizing
from happiestfuntokenizing.happiestfuntokenizing import Tokenizer
import string
from collections import Counter

def tokenize(text: str):
    """Tokenize a single text string into a list of tokens"""
    tokenizer = Tokenizer()
    tokens = tokenizer.tokenize(text)
    punctuation = set(string.punctuation)
    tokens = [token for token in tokens if token not in punctuation]
    return tokens

# def lower_and_remove_punctuation(text: str):
#     """Lowercase text and remove punctuation for text column for Roberta"""
#     text = text.lower()
#     punctuation = set(string.punctuation)
#     text = ''.join([char for char in text if char not in punctuation])
#     return text

def find_stopwords(control_dataset):
    """Find top 100 words from the control dataset to use as stop words."""

    all_tokens = []
    for tokens in (control_dataset['tokens'].tolist()):
        all_tokens.extend(tokens)

    freq = Counter(all_tokens)
    top_100_words = [word for word, _ in freq.most_common(100)]

    return top_100_words

In [ ]:
# Load preprocessed data from Google Drive
with open(f'{FOLDER}/pre_depression.pkl', 'rb') as f:
    pre_depression = pickle.load(f)

with open(f'{FOLDER}/pre_control.pkl', 'rb') as f:
    pre_control = pickle.load(f)

In [ ]:
# Testing if preprocessed data is loaded

print(pre_depression['tokens'].head())
print()
print(pre_control['tokens'].head())
print(pre_control['text'].head())

20    [trying, hi, sorry, writing, bad, headache, di...
39    [friend, blanking, feels, months, complete, sh...
67    [study, hall, social, anxiety, bruh, study, ha...
72                 [positive, thoughts, happy, publish]
79    [starting, blowup, mattress, today, very, “, d...
Name: tokens, dtype: object

315                         [man, love, bandicoot, crash]
651     [pc, 700-750, budget, gaming, high, ultra, set...
730     [price, gpus, down, bitcoin, mining, trying, s...
1354    [our, service, available, area, hey, totally, ...
1598                                                [wow]
Name: tokens, dtype: object
315              Man, I do love me some Bandicoot crash. 
651     How good is this PC for my 700-750$ budget? Wa...
730     When is the price of gpus going down? I know t...
1354    Our service is not available in your area. Hey...
1598                                                 Wow 
Name: text, dtype: object


### Preprocessing (only need to run once)

In [ ]:
def preprocess(depression_data, control_data):
    """Preprocess the depression and control datasets by tokenizing and removing stopwords"""
    control_data = control_data.copy()
    depression_data = depression_data.copy()

    # control_data['text'] = control_data['text'].apply(lower_and_remove_punctuation)
    # depression_data['text'] = depression_data['text'].apply(lower_and_remove_punctuation)

    # tokenize
    control_data['tokens'] = control_data['text'].apply(tokenize)
    depression_data['tokens'] = depression_data['text'].apply(tokenize)

    # stop word removal
    stopwords = find_stopwords(control_data)
    # print(stopwords)
    depression_data['tokens'] = depression_data['tokens'].apply(lambda tokens: [t for t in tokens if t not in stopwords])
    control_data['tokens'] = control_data['tokens'].apply(lambda tokens: [t for t in tokens if t not in stopwords])
    return depression_data, control_data

pre_depression, pre_control = preprocess(df_depression, df_control)

In [ ]:
# Save preprocessed data to Google Drive
with open(f'{FOLDER}/pre_depression.pkl', 'wb') as f:
    pickle.dump(pre_depression, f)

with open(f'{FOLDER}/pre_control.pkl', 'wb') as f:
    pickle.dump(pre_control, f)

## Reddit Topics with LDA

In [ ]:
# Load LDA model
lda_model = LdaModel.load(f'{FOLDER}/lda_model.gensim')
dictionary = Dictionary.load(f'{FOLDER}/dictionary.gensim')

# Load dictionary and corpus
with open(f'{FOLDER}/combined_corpus.pkl', 'rb') as f:
    combined_corpus = pickle.load(f)
with open(f'{FOLDER}/control_corpus.pkl', 'rb') as f:
    control_corpus = pickle.load(f)

### LDA model (run once)

In [ ]:
all_tokens = pre_depression['tokens'].tolist() + pre_control['tokens'].tolist()
# LDA should be trained on the entire dataset(all posts)

# dictionary and corpus
dictionary = Dictionary(all_tokens)
combined_corpus = [dictionary.doc2bow(doc) for doc in all_tokens]
# create a BoW representation for each doc/post, term-document, doc/post represented as (word_id, count)

lda_model = LdaModel(
    combined_corpus,
    num_topics=200,
    id2word=dictionary,
    passes=1,
)

control_corpus = [dictionary.doc2bow(doc) for doc in pre_control['tokens']]

In [72]:
print(dict(list(dictionary.token2id.items())[:10]))
print(combined_corpus[:2])

{'10': 0, 'able': 1, 'air': 2, 'anxiety': 3, 'anymore': 4, 'anywhere': 5, 'backing': 6, 'bad': 7, 'black': 8, 'bottom': 9}
[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 3), (20, 2), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 3), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 3), (46, 2), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 2), (69, 1), (70, 2), (71, 1), (72, 2), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 2), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 2), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98

In [ ]:
# Save LDA model, dictionary, and corpus
lda_model.save(f'{FOLDER}/lda_model.gensim')
dictionary.save(f'{FOLDER}/dictionary.gensim')
with open(f'{FOLDER}/combined_corpus.pkl', 'wb') as f:
    pickle.dump(combined_corpus, f)

# Save control corpus for later use
with open(f'{FOLDER}/control_corpus.pkl', 'wb') as f:
    pickle.dump(control_corpus, f)

## (Distil)RoBERTa Embeddings

In [ ]:
# TODO: My RoBERTa code!

# only 6 transformer blocks (run faster)
# extracting embeddings from the 5th layer for downstream classification
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")
model = AutoModelForMaskedLM.from_pretrained("distilbert/distilroberta-base", output_hidden_states=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm)

In [ ]:
def get_distilroberta_embeddings(sentences, tokenizer, model, device):
    """
    Generate embeddings for a list of sentences using DistilRoBERTa.

    Returns:
    - embeddings: NumPy array of shape (num_sentences, embedding_dim).
    """
    all_embeddings = []
    batch_size = 16 # batches otherwise ram

    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i:i + batch_size]

        inputs = tokenizer(
            batch_sentences,
            return_tensors="pt", # pytorch tensor
            padding=True,
            truncation=True, # truncate sequences > max_length
            max_length=512 # max sequence length
        )

        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            hidden_states = outputs.hidden_states

        # 5th layer's output for embeddings
        selected_layer = hidden_states[5]  # shape: (batch_size, sequence_length, hidden_size)

        # sentence-level representation
        sentence_embeddings = selected_layer.mean(dim=1)  # shape: (batch_size, hidden_size)
        # each word/token has a single embedding representation.
        # To get the embedding for a post, average the embeddings of each token

        all_embeddings.append(sentence_embeddings.cpu())

    all_embeddings = torch.cat(all_embeddings, dim=0).numpy()

    return all_embeddings

## Main

### Set up

In [ ]:
results = {}

# Reference values for LDA and DistilRoBERTa
reference_values = {
    "Anger": {"LDA": 0.794, "DistilRoBERTa": 0.928},
    "Anhedonia": {"LDA": 0.906, "DistilRoBERTa": 0.956},
    "Anxiety": {"LDA": 0.837, "DistilRoBERTa": 0.952},
    "Disordered Eating": {"LDA": 0.905, "DistilRoBERTa": 0.952},
    "Loneliness": {"LDA": 0.806, "DistilRoBERTa": 0.907},
    "Sad Mood": {"LDA": 0.788, "DistilRoBERTa": 0.919},
    "Self-Loathing": {"LDA": 0.815, "DistilRoBERTa": 0.922},
    "Sleep Problems": {"LDA": 0.909, "DistilRoBERTa": 0.956},
    "Somatic Complaints": {"LDA": 0.880, "DistilRoBERTa": 0.925},
    "Worthlessness": {"LDA": 0.700, "DistilRoBERTa": 0.897}
}

In [ ]:
# Prepare Symptom Dataset

# Don't evaluate on Fatigue, Concentration Deficit, Suicidal Thoughts
symptom_to_subreddits = {
    "Anger": ["Anger"],
    "Anhedonia": ["anhedonia", "DeadBedrooms"],
    "Anxiety": ["Anxiety", "AnxietyDepression", "HealthAnxiety", "PanicAttack"],
    # "Concentration Deficit": ["DecisionMaking", "shouldi"],
    "Disordered Eating": ["bingeeating", "BingeEatingDisorder", "EatingDisorders", "eating_disorders", "EDAnonymous"],
    # "Fatigue": ["chronicfatigue", "Fatigue"],
    "Loneliness": ["ForeverAlone", "lonely"],
    "Sad Mood": ["cry", "grief", "sad", "Sadness"],
    "Self-Loathing": ["AvPD", "SelfHate", "selfhelp", "socialanxiety", "whatsbotheringyou"],
    "Sleep Problems": ["insomnia", "sleep"],
    "Somatic Complaints": ["cfs", "ChronicPain", "Constipation", "EssentialTremor", "headaches", "ibs", "tinnitus"],
    # "Suicidal Thoughts": ["AdultSelfHarm", "selfharm", "SuicideWatch"],
    "Worthlessness": ["Guilt", "Pessimism", "selfhelp", "whatsbotheringyou"]
}

symptom_datasets = {}

for symptom, subreddits in symptom_to_subreddits.items():
    # Filter rows where the subreddit matches one of the subreddits for this symptom
    symptom_data = pre_depression[pre_depression['subreddit'].isin(subreddits)]
    symptom_datasets[symptom] = symptom_data

# anxiety_data = symptom_datasets["Anxiety"]
# print(len(anxiety_data))

### LDA evaluation

In [ ]:
def get_topic_distributions(corpus, model, num_topics):
    distributions = []
    for doc in corpus:
        sparse_dist = model[doc]
        dense_dist = np.zeros(num_topics)
        for topic_id, prob in sparse_dist:
            dense_dist[topic_id] = prob
        distributions.append(dense_dist)
    return np.array(distributions)

In [ ]:
def lda_rf_cross_validation():
    """
    It runs 5-fold cross validation with random forest to evaluate my LDA performance.
    """
    # TODO: Print your training and testing scores!
    for symptom, symptom_data in symptom_datasets.items():
        print(f"Running for symptom: {symptom}")
        # --- LDA Workflow ---
        symptom_corpus = [dictionary.doc2bow(doc) for doc in symptom_data['tokens']]

        num_topics = lda_model.num_topics
        X_symptom = get_topic_distributions(symptom_corpus, lda_model, num_topics)
        X_control = get_topic_distributions(control_corpus, lda_model, num_topics)

        X_lda = np.vstack([X_symptom, X_control])
        y_lda = np.hstack([np.ones(X_symptom.shape[0]), np.zeros(X_control.shape[0])])

        rf_classifier_lda = RandomForestClassifier(n_estimators=100, random_state=42)

        cv_lda = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_results_lda = cross_validate(rf_classifier_lda, X=X_lda, y=y_lda, cv=cv_lda, scoring='roc_auc', return_train_score=True)
        mean_train_auc_lda = round(cv_results_lda['train_score'].mean(), 3)
        mean_test_auc_lda = round(cv_results_lda['test_score'].mean(), 3)

        print("Training AUC Scores:", cv_results_lda['train_score'])
        print("Testing AUC Scores:", cv_results_lda['test_score'])
        print(f"Mean Training AUC: {mean_train_auc_lda:.3f}")
        print(f"Mean Testing AUC: {mean_test_auc_lda:.3f}")
        print("\n")

        if symptom not in results:
            results[symptom] = {"Symptom": symptom}

        results[symptom].update({
            "LDA Test AUC": mean_test_auc_lda,
            "LDA Ref AUC": reference_values.get(symptom, {}).get("LDA", None)
        })

lda_rf_cross_validation()

Running for symptom: Anger
Training AUC Scores: [0.99722616 0.99794291 0.99795207 0.99798043 0.99745202]
Testing AUC Scores: [0.94613198 0.91483141 0.89858548 0.9302259  0.92884294]
Mean Training AUC: 0.998
Mean Testing AUC: 0.924


Running for symptom: Anhedonia
Training AUC Scores: [0.99924015 0.99925321 0.99918922 0.99921041 0.999166  ]
Testing AUC Scores: [0.96119557 0.95406614 0.95451698 0.95592301 0.95632099]
Mean Training AUC: 0.999
Mean Testing AUC: 0.956


Running for symptom: Anxiety
Training AUC Scores: [0.99931083 0.99923988 0.99936218 0.99943825 0.99941006]
Testing AUC Scores: [0.93869649 0.9408955  0.93631066 0.94088176 0.93437597]
Mean Training AUC: 0.999
Mean Testing AUC: 0.938


Running for symptom: Disordered Eating
Training AUC Scores: [0.99890382 0.99893785 0.99893579 0.99894196 0.99892368]
Testing AUC Scores: [0.97520155 0.96861908 0.96244412 0.95815335 0.96508473]
Mean Training AUC: 0.999
Mean Testing AUC: 0.966


Running for symptom: Loneliness
Training AUC Score

### DistilRoBERTa Evaluation

In [ ]:
CACHE_FILE = os.path.join(FOLDER, 'embeddings_cache.npz')

def save_embeddings_to_cache(file_path, embeddings_dict):
    """Save embeddings to a .npz file."""
    np.savez(file_path, **embeddings_dict)

def load_embeddings_from_cache(file_path):
    """Load embeddings from a .npz file."""
    if os.path.exists(file_path):
        return dict(np.load(file_path, allow_pickle=True))
    return {}

In [ ]:
def drb_rf_cross_validation():
    """
    It runs 5-fold cross validation with random forest to evaluate my DistilRoBERTa performance.
    """
    embeddings_cache = load_embeddings_from_cache(CACHE_FILE)
    updated_cache = {}

    for symptom, symptom_data in symptom_datasets.items():
        print(f"Running for symptom: {symptom}")
        # --- DistilRoBERTa Workflow ---

        # generate or load embeddings for depression and control datasets
        if symptom in embeddings_cache:
            symptom_embeddings = embeddings_cache[symptom]
        else:
            symptom_embeddings = get_distilroberta_embeddings(symptom_data['text'].tolist(), tokenizer, model, device)
            updated_cache[symptom] = symptom_embeddings

        if "control" in embeddings_cache:
            control_embeddings = embeddings_cache["control"]
        else:
            control_embeddings = get_distilroberta_embeddings(pre_control['text'].tolist(), tokenizer, model, device)
            updated_cache["control"] = control_embeddings

        # Create labels
        symptom_labels = [1] * len(symptom_data)
        control_labels = [0] * len(pre_control)

        X_roberta = np.vstack([symptom_embeddings, control_embeddings])
        y_roberta = np.array(symptom_labels + control_labels)
        rf_classifier_roberta = RandomForestClassifier(n_estimators=100, random_state=42)
        cv_roberta = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_results_roberta = cross_validate(rf_classifier_roberta, X=X_roberta, y=y_roberta, cv=cv_roberta, scoring='roc_auc', return_train_score=True)

        mean_train_auc_roberta = round(cv_results_roberta['train_score'].mean(), 3)
        mean_test_auc_roberta = round(cv_results_roberta['test_score'].mean(), 3)

        print("DistilRoBERTa Training AUC:", cv_results_roberta['train_score'])
        print("DistilRoBERTa Testing AUC:", cv_results_roberta['test_score'])
        print(f"DistilRoBERTa Mean Training AUC: {mean_train_auc_roberta:.3f}")
        print(f"DistilRoBERTa Mean Testing AUC: {mean_test_auc_roberta:.3f}")
        print("\n")

        if symptom not in results:
            results[symptom] = {"Symptom": symptom}
        results[symptom].update({
            "DRB Test AUC": mean_test_auc_roberta,
            "DRB Ref AUC": reference_values.get(symptom, {}).get("DistilRoBERTa", None)
        })
    save_embeddings_to_cache(CACHE_FILE, {**embeddings_cache, **updated_cache})

drb_rf_cross_validation()

Running for symptom: Anger
DistilRoBERTa Training AUC: [1. 1. 1. 1. 1.]
DistilRoBERTa Testing AUC: [0.96075626 0.95707611 0.9322941  0.94363149 0.92407363]
DistilRoBERTa Mean Training AUC: 1.000
DistilRoBERTa Mean Testing AUC: 0.944


Running for symptom: Anhedonia
DistilRoBERTa Training AUC: [1. 1. 1. 1. 1.]
DistilRoBERTa Testing AUC: [0.96647164 0.94914559 0.95954764 0.9590752  0.96027029]
DistilRoBERTa Mean Training AUC: 1.000
DistilRoBERTa Mean Testing AUC: 0.959


Running for symptom: Anxiety
DistilRoBERTa Training AUC: [1. 1. 1. 1. 1.]
DistilRoBERTa Testing AUC: [0.9506506  0.96061159 0.96026363 0.95526611 0.95481852]
DistilRoBERTa Mean Training AUC: 1.000
DistilRoBERTa Mean Testing AUC: 0.956


Running for symptom: Disordered Eating
DistilRoBERTa Training AUC: [1. 1. 1. 1. 1.]
DistilRoBERTa Testing AUC: [0.96263582 0.9548989  0.94647068 0.94971045 0.95930296]
DistilRoBERTa Mean Training AUC: 1.000
DistilRoBERTa Mean Testing AUC: 0.955


Running for symptom: Loneliness
DistilRoBE

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

results_df = pd.DataFrame(list(results.values()))

print(results_df)

              Symptom  LDA Test AUC  LDA Ref AUC  DRB Test AUC  DRB Ref AUC
0               Anger         0.924        0.794         0.944        0.928
1           Anhedonia         0.956        0.906         0.959        0.956
2             Anxiety         0.938        0.837         0.956        0.952
3   Disordered Eating         0.966        0.905         0.955        0.952
4          Loneliness         0.845        0.806         0.916        0.907
5            Sad Mood         0.847        0.788         0.935        0.919
6       Self-Loathing         0.867        0.815         0.934        0.922
7      Sleep Problems         0.977        0.909         0.958        0.956
8  Somatic Complaints         0.914        0.880         0.931        0.925
9       Worthlessness         0.760        0.700         0.924        0.897
